In [ ]:
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg, Robot
import cv2
import numpy as np
from PID import PID

# Camera
camera_width = 640
camera_height = 480
camera = Camera.instance(width=camera_width, height=camera_height)

# Robot
robot = Robot()

# PID
steering_pid = PID(Kp=0.003, Ki=0.0, Kd=0.0005, setpoint=camera_width/2)

# Widgets
grayscale_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)
line_mask_widget = widgets.Image(format='jpeg', width=camera_width, height=camera_height)

# Start/Stop Button
robot_stop = True
stop_button = widgets.Button(description="STOP", button_style='danger', layout=widgets.Layout(width='100px', height='50px'))
start_button = widgets.Button(description="START", button_style='success', layout=widgets.Layout(width='100px', height='50px'))
def stop_robot(button):
    global robot_stop
    robot_stop = True
    robot.stop()
def start_robot(button):
    global robot_stop
    robot_stop = False
stop_button.on_click(stop_robot)
start_button.on_click(start_robot)

# Display Widgets
display(widgets.VBox([
    widgets.HBox([grayscale_widget, line_mask_widget]),
    widgets.HBox([start_button, stop_button])]))

# Callback function once the camera frame is available
def follow_black_line(frame):
    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    grayscale_display_frame = cv2.cvtColor(grayscale_frame, cv2.COLOR_GRAY2BGR)
    grayscale_widget.value = bgr8_to_jpeg(grayscale_display_frame)

    # Apply Otsu's threshold (invert so black line becomes white)
    _, binary_line_mask = cv2.threshold(grayscale_frame, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    binary_line_mask_bgr = cv2.cvtColor(binary_line_mask, cv2.COLOR_GRAY2BGR)
    line_mask_widget.value = bgr8_to_jpeg(binary_line_mask_bgr)

    # Crop the bottom part of the frame (near the robot) for line detection
    frame_height, frame_width = binary_line_mask.shape
    crop_height = frame_height // 4
    cropped_mask = binary_line_mask[frame_height - crop_height : frame_height, :]

    moments = cv2.moments(cropped_mask)
    if moments['m00'] > 0:
        centroid_x = int(moments['m10'] / moments['m00'])
        centroid_y = int(moments['m01'] / moments['m00'])

        # Visualize the centroid
        cv2.circle(binary_line_mask_bgr, (centroid_x, frame_height - crop_height + centroid_y), 5, (0,0,255), -1)
        line_mask_widget.value = bgr8_to_jpeg(binary_line_mask_bgr)

        # PID Steering
        forward_speed = 0.2
        steering_adjust = steering_pid.update(centroid_x)

        left_motor_speed = forward_speed - steering_adjust
        right_motor_speed = forward_speed + steering_adjust

        left_motor_speed = max(min(left_motor_speed, 1.0), -1.0)
        right_motor_speed = max(min(right_motor_speed, 1.0), -1.0)

        global robot_stop
        if robot_stop:
            left_motor_speed = 0
            right_motor_speed = 0

        robot.set_motors(left_motor_speed, right_motor_speed)
    else:
        robot.stop()

# Link camera frames to processing
camera.observe(lambda change: follow_black_line(change['new']), names='value')